In [33]:
from geopy.geocoders import Nominatim
import json
import folium

## Load Clubs data

In [106]:
with open("../scraper/data/clubs.json", "r") as in_file:
    clubs = json.load(in_file)

## Retrieve coordinates using OpenStreetMap API

In [48]:
for club in clubs:
    if "position" not in club:
        location = geolocator.geocode(club["stadium"])
        if location is not None:
            club["position"] = (location.latitude, location.longitude)

In [51]:
for club in clubs:
    if "position" not in club:
        location = geolocator.geocode(club["stadium"] + " " + club["name"])
        if location is not None:
            club["position"] = (location.latitude, location.longitude)

## Retrieve missing coordinates using GoogleMap requests

In [121]:
import requests

GOOGLE_MAPS_API_URL = 'http://maps.googleapis.com/maps/api/geocode/json'


for club in clubs:
    if "position" not in club:
        
        params = {
            'address': club["stadium"]
        }
        # Do the request and get the response data
        req = requests.get(GOOGLE_MAPS_API_URL, params=params)
        res = req.json()
        # Use the first result
        if len(res['results']) > 0:
            result = res['results'][0]
            club["position"] = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])

In [109]:
import requests

GOOGLE_MAPS_API_URL = 'http://maps.googleapis.com/maps/api/geocode/json'

for club in clubs:
    if "position" not in club:
        
        params = {
            'address': club["stadium"] + " " + club["name"]
        }
        # Do the request and get the response data
        req = requests.get(GOOGLE_MAPS_API_URL, params=params)
        res = req.json()
        # Use the first result
        if len(res['results']) > 0:
            result = res['results'][0]
            club["position"] = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])

### Manual clean up

In [114]:
for i in range(len(clubs)):
    if "position" not in clubs[i]:
        print(str(i) + ": " + clubs[i]["name"])

93: Excelsior Rotterdam
170: Medipol Basaksehir
208: KSC Lokeren
209: KAS Eupen
221: PFK Stal Kamyanske
235: FC Vysocina Jihlava


In [149]:
# Change manually the three stadium that couldn't be found with OpenStreetMap nor GoogleMaps
# We then rerun the google map coordinate retrieval
clubs[93]["stadium"] = 'S.B.V. Excelsior'
clubs[170]["stadium"] = 'Başakşehir Fatih Terim Stadium'
clubs[208]["stadium"] = 'Lokeren Daknamstadion'
clubs[209]["stadium"] = 'KAS Eupen'
clubs[221]["stadium"] = 'Obolon Arena'
clubs[235]["stadium"] = 'FC Vysočina Jihlava'
clubs[20]["stadium"] = "Juventus Stadium"

In [148]:
for i in range(len(clubs)):
    if 'Mouscron' in clubs[i]["name"]:
        print(i)

201


In [150]:
params = {
    'address': clubs[201]["stadium"] + " " + clubs[201]["name"]
}
# Do the request and get the response data
req = requests.get(GOOGLE_MAPS_API_URL, params=params)
res = req.json()
# Use the first result
if len(res['results']) > 0:
    result = res['results'][0]
    clubs[201]["position"] = (result['geometry']['location']['lat'], result['geometry']['location']['lng'])

## TODO: Move slighly clubs which plays in the same stadium so we can see them both on the map

## Save/Load Clubs with location data

In [160]:
'''
with open("../scraper/data/clubs_with_location.json", "w") as out:
    json.dump(clubs, out)
'''

'\nwith open("../scraper/data/clubs_with_location.json", "w") as out:\n    json.dump(clubs, out)\n'

In [159]:
with open("../scraper/data/clubs_with_location.json", "r") as in_file:
    clubs = json.load(in_file)

# Display clubs on map

In [151]:
m = folium.Map(location=[52, 15], zoom_start=4)

In [152]:
for club in clubs:
    folium.Marker(club["position"], popup=club["name"]).add_to(m)

In [153]:
m